# How to resolve an image CAPTCHA system with Convolutional Neural Networks (CNNs)?

## Goals
By the end of this case, you will be familiar with convolutional neural network. Specifically, you will learn how to add convolutional layers, flatten layers, Pooling layers, early stopping in your model, and save and load models in TensorFlow to improve performance in the training of the network.

## Introduction

**Business Context.** In the process of performing web scrapping in order to obtain data from web pages, companies use capchtas in order to avoid the automation and extraction of information through Bots. There are many types of captchas, one of the first and most used are image captchas, where the user is asked to select one of the images corresponding to a statement. Ex: Select shirts from the list of images below.

<img src="data/images/captcha.png" width="500" height="500" />


**Business Problem.** As a computer vision engineer at an web scrapping company, you must **create a model than can break images capchta**

**Analytical Context.** The data is downloaded from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/fashion_mnist) and it is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes (Bag, Coat, Dress, Pullover, Sandal, Shirt, Sneaker, T-shirt/top, Trouser). If you want to explore the Fashion Mnist dataset click [here](https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=fashion_mnist)

The case is structured as follows, you will:
 1. Create a simple neural network.
 2. Learn the basics of convolutional and pooling labels.
 2. Create a convolutional Neural network
 3. Apply early stopping.
 4. Save and Load the model in TF.

## Reviewing basics of neural networks

So far, we've learned about neural networks and how they can be really useful when dealing with large structured datasets exhibiting highly nonlinear relationships. But in order to be able to handle more complex data like images or video, it's necessary to introduce a particular type of neural network architecture, the **convolutional neural network**.

Remember that a neural network is a series of algorithms that endeavors to recognize underlying relationships in a set of data. The inner workings of the algorithm takes inputs on the left-hand side to provide outputs on the right-hand side by multiplying each layer by the edges between neurons to get the next layer. Then, the algorithm uses multivariable calculus (in a process known as **gradient descent**) to optimize each edge:

<img src="data/images/neuralnetworkdiagram.png" width="500" height="500" />

How do we make good or bad choices within the network? We compare the outputs of the predictions (using the loss function) and make tiny changes to compare the outputs. Most frequently, we use a learning rate and a gradient descent method to estimate the changes that our successive models have used.

Before proceeding, make sure you are familiar with the following terms:

1. **Optimizer**: The optimization algorithm used to update weights in neural network training. A common optimizer is **stochastic gradient descent (SGD)**
1. **Learning rate**: The influence of the gradient in updating the weights of the neural network at each step. Smaller values correspond to smaller (slower) updates; the default in `SGD` of `keras` is 0.01
1. **Loss function**: The function that determines how to evaluate predictions from the model against the labeled data. A common loss function is [mean squared error](https://keras.io/api/losses/regression_losses/#mean_squared_error-function).
1. **Epochs**: The number of cycles that are run on the full training dataset

## Let's build one!

The Python library typically used to build a neural network is either Tensorflow or Pytorch. For this bootcamp, we will use Tensorflow in order to classify the fashion mnist dataset, which is a dataset of 60,000 28x28 grayscale images of 10 fashion categories, along with a test set of 10,000 images. Dataset come from [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/fashion_mnist/load_data).
 

In [ ]:
# import required libraries 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import matplotlib.image as img
from scipy import signal

In [ ]:
# donwload the dataset that will be used later.
(X_train, Y_train ), (X_test, Y_test ) = datasets.fashion_mnist.load_data()

Convolutional neural networks have a higher computational cost, for that reason and to carry out this practical example we are going to take less data in order to be able to perform the processing from the server.

In [ ]:
# Use less data 
train_images = X_train[:30000,:,:]
test_images = X_test[:5000,:,:]
train_labels = Y_train[:30000]
test_labels  = Y_test[:5000]

In [ ]:
# Check if the data is not balanced.
np.unique(test_labels, return_counts=True, return_index=True)

### Verifying the images

To check that the dataset is OK, we are going to display the first 20 images from the training set as well as the class that each one belongs to:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(10,10))
for i in range(20):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
# Normalize pixel values to be between 0 and 1
train_image = train_images / 255.0
test_image = test_images / 255.0

## Dense and Flatten
Dense Layers: It's a regular fully connected Neural Network layer. It is used without parameters, you have to select the units (Positive integer)
Flatten Layer: It converts a matrix to single array.

<img src="data/images/flatten.png" width="400" height="400" />

In this case, FASHION_MNIST has 10 output classes, so you use a final Dense layer with 10 outputs and a softmax activation function.

In [ ]:
# Create the sequential model. 
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [ ]:
# Summary of the architecture of the neural network.
model.summary()

The first model in a regular server took 145 [s] to train.

In [ ]:
# Compile and fit the model.
start = time.time()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_image, train_labels, epochs=10, validation_split = 0.10)

print("Time training: {}".format(time.time()-start))

In [ ]:
df = pd.DataFrame(history.history)
df.plot(y=['accuracy', 'val_accuracy'])

In [ ]:
df = pd.DataFrame(history.history)
df.plot(y=['loss', 'val_loss'])

In [ ]:
test_loss, test_acc = model.evaluate(test_image,  test_labels, verbose=2)
print('accuracy: {acc:0.3f}'.format(acc=test_acc))

## But how do you make that even better? What is convolution?

In mathematics, **convolution** is an operation on two functions that produces a third function expressing how the shape of one is modified by the other. Our particular interest is to use a convolution that extracts patterns from an image. You probably know how an image can be represented as a large matrix of numbers (pixels). A convolution consists of passing a smaller matrix, called a **convolutional filter** (or **kernel**), over this matrix. You can find more examples of filters [here](https://en.wikipedia.org/wiki/Kernel_(image_processing))  .Here is a demonstration of how the filter is applied with a kernel [1 0 1; 0 1 0; 1 0 1]:

<img src=data/images/2D_Convolution_Animation.gif width="500">

Source: [Wikipedia](https://commons.wikimedia.org/wiki/File:2D_Convolution_Animation.gif)

Convolution is important for image handling because it allows us to extract features from it. This means that by using pooling and convolution, we will be able to reduce the dimensions of the data we are processing for prediction or modeling.
In this case we are going to use a Max pooling operation for 2d spatial data, to underestand the pooling function, let's
see the next images:

<img src=data/images/Pooling-DeepLearningSpecialization.png width="600">

As you can see the Pooling function helps to reduce the dimensionality of the images by compressing the images and take the higher value per matrix.

## What is a CNN?

A **convolutional neural network (CNN)** is a neural network-based deep learning algorithm which, when applied to an input image, assigns importance (learnable weights and biases) to various aspects/objects in the image to differentiate them from each other.

For example, if our original input images were of dimensions 200x200x3 (width, height, and depth, respectively), where the depth refers to the 3 RGB color channels, the neurons in a layer would only be connected to a small region of the layer before it, instead of all of the neurons:

<img src=data/images/CNN_Main.png width="1000">


Here is how we can perform convolution in Python:

In [ ]:
# Read an image 
imageMat = img.imread('data/images/goku.png')

# Shape of the image
print("Image shape:", imageMat.shape) 

# Reshape for just take one channel.
imageMat_reshape = imageMat.reshape(imageMat.shape[0], -1)

all_imgs = np.asfarray(imageMat_reshape[:, 1:])

# Reshape
img = all_imgs.reshape(200, 799)

# Filters
conv_filter = np.array([[1,1,1,1], [1,1,1,1], [1,1,1,1], [1,1,1,1]])
conv_filter2 = np.array([[1,1,1,1], [0,0,0,0], [0,0,0,0], [-1,-1,-1,-1]])

image_new = signal.fftconvolve(img, conv_filter)
image_new2 = signal.fftconvolve(img, conv_filter2)

# Images
fig, axes = plt.subplots(1, 3)
axes[0].imshow(imageMat, cmap=plt.cm.gray)
axes[0].set_title("Original", fontsize=13)
axes[0].axis("off")

axes[1].imshow(image_new, cmap=plt.cm.gray)
axes[1].set_title("Convolution 1", fontsize=13)
axes[1].axis("off")

axes[2].imshow(image_new2, cmap=plt.cm.gray)
axes[2].set_title("Convolution 2", fontsize=13)
axes[2].axis("off")

## Python implementation of a CNN

We are going to use the `tensorflow` package (`tensorflow` is an end-to-end open source platform for machine learning) to implement a CNN in Python; for further investigation, check [here](https://www.tensorflow.org/).

### Convolutional layers

The following code defines the convolutional stack of `Conv2D` and `MaxPooling2D` layers.

A CNN takes an input of shape (`image_height`, `image_width`, `color_channels`). In this example, we will configure a CNN to process inputs of shape (28, 28, 1), which is the format of FASHION MNIST images.

You'll notice that there's a bit of a change here in that the training data needed to be reshaped. That's because the first convolution expects a single tensor containing everything, so instead of 30,000 28x28x1 items in a list, we have a single 4D list that is 30,000x28x28x1, and the same for the test images. If you don't do this, you'll get an error when training as the Convolutions do not recognize the shape.

In [ ]:
# reshape dataset to have a single channel
# batch_shape + (cols, rows,channels)
train_img = train_images.reshape(30000, 28, 28, 1)
train_img = train_img / 255.0
test_img = test_images.reshape(5000, 28, 28, 1)
test_img = test_img/255.0

But wait ... what filters is my conv2D function using?
The same network calculates them when it performs backpropagation, it finds the ideal value of the filters, you only have to define the amount of filters you want to use, and how you want to initialize it. Con2D by default initializes the weights with glorot uniform for more info [here](https://www.tensorflow.org/api_docs/python/tf/keras/initializers/GlorotUniform)

In [ ]:
# Create the sequential model with convolutional layers.
model2 = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])


In [ ]:
# Summary of the architecture of the neural network.
## Capa convolucional
# ((shape of width of the filter * shape of height of the filter * number of filters in the previous layer+1)*number of filters)
# (3*3*1+1)*32
## Capas totalmente conectadas
# (Current layers neurons * previus layers neurons) + 1*c
# (128*5408) + 1*128
# (10*128) + 1*10
model2.summary()

### Why do we have one less output size when it goes through the convolutional layer? 

The main reason is that the filter takes the pixels from the edges reducing the size of the data, in the case of a 3x3 convolutional filter, we are going to lose 2 pixels in x, y. In the case of a 5x5 filter, we are going to lose 4 pixels in x, y.

<img src=data/images/Reduce-covolutional.png width="550">

In [ ]:
# Compile and fit the model.
start = time.time()
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history2 = model2.fit(train_img, train_labels, epochs=5, validation_split = 0.10)
print("Time training: {}".format(time.time()-start))

The second model in a regular server took 865 [s] to train.

In [ ]:
df = pd.DataFrame(history2.history)
df.plot(y=['accuracy', 'val_accuracy'])

In [ ]:
df = pd.DataFrame(history2.history)
df.plot(y=['loss', 'val_loss'])

In [ ]:
test_loss, test_acc = model2.evaluate(test_img,  test_labels, verbose=2)
print('accuracy: {acc:0.3f}'.format(acc=test_acc))

### Early Stopping
The callbacks allow us in each interaction to execute a specific function, in this case every time an epoch ends it calls the callback_acc function to check what the accuracy was in that last epoch, if the accuracy is greater than the condition, it stops training immediately.  **Early stopping** is a way to stop training when a specific situation occurs, it helps us to avoid overfitting

In [ ]:
class callback_acc(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.90):
          print("\nReached at least 90% accuracy..")
          self.model.stop_training = True

I recommend starting with filters in the range [32, 64, 128] in the earlier and increasing up to [256, 512, 1024] in the deeper layers.

In [ ]:
# Create the sequential model with convolutional layers.
model3 = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(2, 2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
])
    
# Create the callback function
callbacks = callback_acc()

In [ ]:
# Compile and fit the model.
start = time.time()
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history3 = model3.fit(train_img, train_labels, epochs=5, validation_split = 0.10, callbacks=[callbacks])
print("Time training: {}".format(time.time()-start))

The third model in a regular server took 530 [s] to train.

In [ ]:
df = pd.DataFrame(history3.history)
df.plot(y=['accuracy', 'val_accuracy'])

In [ ]:
df = pd.DataFrame(history3.history)
df.plot(y=['loss', 'val_loss'])

In [ ]:
test_loss, test_acc = model3.evaluate(test_img,  test_labels, verbose=2)
print('accuracy: {acc:0.3f}'.format(acc=test_acc))

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

In [ ]:
# Make predictions
predictions = model3.predict(test_img)

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_img)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

## Save and Load Models

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share. More info about save and load models you can visit [Tensorflow](https://www.tensorflow.org/tutorials/keras/save_and_load). 

In [ ]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
model3.save('saved_model/model_ds4a')

In [ ]:
model_load = models.load_model('saved_model/model_ds4a')

# Check its architecture
model_load.summary()

In [ ]:
# Choose a random test image
random_inx = np.random.choice(test_img.shape[0])
inx = 30
test_image = test_images[inx]
plt.imshow(test_image)
plt.show()

In [ ]:
# Get the model predictions and Evaluate the restored model
predictions = model_load.predict(test_image[np.newaxis,...,np.newaxis])
print(f"Model Prediction:{class_names[np.argmax(predictions)]}")

## Conclusions

In this case, we created our first convolutional neural network to break images CAPTCHAS. We discovered the power of the convolutional layers and pooling layers. Besides, we learned about early stopping and save models to using later in our productions apps. 

## Takeaways

In this case, we introduced the concept of **Convolutional neural network** and how it can be used as a **classifier**. We did the next process:

 1. Creating a simple neural network.
 2. Learning the basics of convolutional and pooling labels.
 2. Creating a convolutional Neural network
 3. Applying early stopping.
 4. Saving and loading the model in TF.

A neural network is not always the first and best option to use, we recommend trying other models before use this, for example, regression, KNN, SVM, decision three and other models.

A convolutional neural network works well when you are working with images, videos, chat voices, and other not structure data. 

Keep working hard and creating models.

## Atribution.

- [File: 2D Convolution Animation](https://commons.wikimedia.org/wiki/File:2D_Convolution_Animation.gif)